In [1]:
## Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as shp
from random import sample
import requests
import polyline
import folium
from math import radians, sin, cos, atan2, sqrt

meters2miles = 0.000621371

def sample_point(data, crs):
    ## sample one random row from data
    sample_road_row = sample(range(len(data)), 1)

    ## Filter down data to sample road
    sample_road_data = data.loc[sample_road_row].reset_index(drop = True)

    ## Randomly select a pair of coordinates within the road we've selected as a starting point
    sample_road_coords = list(sample_road_data.geometry[0].coords)
    sample_index = sample(range(len(sample_road_coords)), 1)
    sample_geo = shp.Point(list(sample_road_coords[sample_index[0]]))

    ## Return a list with the corresponding sample road row number in 'data' and a geometry object
    return({
        "road_row_num" : sample_road_data.record[0], 
        "point" : gpd.GeoDataFrame({'geometry' : [sample_geo]}, crs = crs)
    })

def get_start_point(data, crs):

    # filter only roadways of level 3
    valid_startpoints = data[data["level"] == 3].reset_index(drop = True)

    return sample_point(valid_startpoints, crs)

def get_end_point(data, start_pos, route_dist, crs):
    
    # calculate distances from start_position, these are in meters since we are using EPSG:32613
    data['distances'] = data['geometry'].distance(start_pos['point'].geometry[0])
    
    # filter only distances that are further than half route_dist
    valid_endpoints = data[data['distances'] >= (route_dist / 2)].reset_index(drop = True)

    return sample_point(valid_endpoints, crs)

def get_route(pickup, dropoff, crs):
    pickup_lon = pickup['point'].to_crs(crs).iloc[0]['geometry'].x
    pickup_lat = pickup['point'].to_crs(crs).iloc[0]['geometry'].y
    
    dropoff_lon = dropoff['point'].to_crs(crs).iloc[0]['geometry'].x
    dropoff_lat = dropoff['point'].to_crs(crs).iloc[0]['geometry'].y
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

def get_map(route, zoom_level):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=zoom_level)
    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = (sin(dlat / 2) * sin(dlat / 2) +
         cos(radians(lat1)) * cos(radians(lat2)) *
         sin(dlon / 2) * sin(dlon / 2))
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    d = radius * c

    return d


- project from WSG84 onto UTM CRS https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system. This then means coordinates are expressed in meters. This will not work if LINESTRING and POINT are in different UTM zones
- Colorado is split up into three reference zones: North, Central and South (https://spatialreference.org/ref/?search=colorado).  However, UTM zone 13N covers all of Colorado but it is in meters.
- then it's exceptionally simple to use https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.distance.html

In [2]:
# load Colorado roadways using EPSG:32613 – WGS 84 / UTM zone 13N (in meters)
CO_roads = gpd.read_file('../data/all_roads.shp').to_crs("EPSG:32613") 

In [3]:
start_pos = get_start_point(CO_roads, "EPSG:32613") # epsg -> crs
end_pos = get_end_point(CO_roads, start_pos, 250 / meters2miles, "EPSG:32613")

In [4]:
print("Distance from ({:.4f},{:.4f}) to ({:.4f},{:.4f}) is {:.2f} miles.".format(
    start_pos['point'].to_crs(4326).iloc[0]['geometry'].y,
    start_pos['point'].to_crs(4326).iloc[0]['geometry'].x,
    end_pos['point'].to_crs(4326).iloc[0]['geometry'].y,
    end_pos['point'].to_crs(4326).iloc[0]['geometry'].x,
    end_pos['point'].distance(start_pos['point']).iloc[0]*meters2miles))
# print(start_pos['road_row_num'])
# print(end_pos['road_row_num'])
# print(CO_roads.loc[start_pos['road_row_num']]['geometry'])
# print(CO_roads.loc[end_pos['road_row_num']]['geometry'])

Distance from (39.7548,-105.0767) to (37.2677,-107.1023) is 203.60 miles.


Something is still not correct. The distance  (in miles) does not correspond to what Google Maps provides.  On the positive side, it seems like a minimum route distance of 250 miles round trip is maintained.

In [5]:
test_route_1 = get_route(start_pos, end_pos, 4326)

In [6]:
get_map(test_route_1, 9)